# SSDの学習の流れ

## 1. 画像のリサイズ
300 x 300にリサイズする

## 2. 画像をネットワークに入力
モデルはvgg、extras、loc、confで構成される

### loc
SSDはバウンディングボックスの走査などの処理はせずに、様々なアスペクト比を持つデフォルトボックスと呼ばれる矩形を用意する(8743個)  
このデフォルトボックスは元のデフォルトボックスの中心から差分を学習する。  
こうすることでデフォルトボックスが物体の中心に寄るよう学習される

学習前のデフォルトボックスの位置  
<img src="https://avinton.com/wp-content/uploads/2018/03/11-1.png" width="500"/>  

学習後のデフォルトボックスの位置  
<img src="https://avinton.com/wp-content/uploads/2018/03/bl.png" width="500" />  

学習前から学習後ではデフォルトの位置が中心に寄ることが分かる（点線が元）  
（中心位置だけではなく高さと幅の差分も計算するのでそこから、ボックスの大きさも変わるっぽい）  
<img src="https://avinton.com/wp-content/uploads/2018/03/8.png" width="500" />


<img src="https://cdn-ak.f.st-hatena.com/images/fotolife/k/kawam0t0/20191223/20191223123247.png" width="600">

### conf
デフォルトボックス毎にどのクラスが含まれているかを学習する  
あとで、どのデフォルトボックスを最終的に残すかの判断に使用される

参考
- [【機械学習】一般物体検知アルゴリズム SSD : 第1編](https://avinton.com/blog/2018/03/single-shot-multibox-detector-explained1/)

In [1]:
import torch
import torch.nn as nn

# vggアーキテクチャ

In [2]:
def make_vgg():
    '''
    Returns：
      (nn.ModuleList): vggのモジュール(部品)のリスト
    '''
    layers = []      # モジュールを格納するリスト
    in_channels = 3  # チャネル数はRGBの3値

    # vggに配置する畳み込み層のフィルター数(チャネル数に相当)
    # 'M''MC'はプーリング層を示す
    cfg = [64, 64, 'M',         # vgg1
           128, 128, 'M',       # vgg2
           256, 256, 256, 'MC', # vgg3
           512, 512, 512, 'M',  # vgg4
           512, 512, 512        # vgg5
           ]
    # vgg1～vgg5の畳み込み層までを生成
    for v in cfg:
        # vgg1、vgg2、vgg4のプーリング層
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, # ウィンドウサイズ2×2
                                    stride=2)]     # ストライド2
        # vgg3のプーリング層
        elif v == 'MC':
            # vgg3のプーリングで(75, 75)の特徴量マップを半分のサイズにする際に、
            # ceil_modeをTrueにすることで75/2=37.5を切り上げて38にする
            # この結果、vgg3のプーリング層から出力される特徴量マップのサイズは
            # (38, 38)になる
            layers += [nn.MaxPool2d(kernel_size=2, # ウィンドウサイズ2×2
                                    stride=2,      # ストライド2
                                    ceil_mode=True)]
        # vgg1～vgg5の畳み込み層
        else:
            conv2d = nn.Conv2d(in_channels,  # 入力時のチャネル数
                               v,            # 出力時のチャネル数(フィルター数)
                               kernel_size=3,# フィルターサイズ3×3
                               padding=1)    # パディングのサイズは1
            
            # 畳み込み層に活性化関数ReLUをセットしてlayersに追加
            # inplace=TrueにするとReLUへの入力値は保持されない(メモリ節約)
            layers += [conv2d, nn.ReLU(inplace=True)]
            # チャネル数を出力時のチャネル数(フィルター数)に置き換える
            in_channels = v
            
    # vgg5のプーリング層
    pool5 = nn.MaxPool2d(kernel_size=3, # ウィンドウサイズ3×3
                         stride=1,      # ストライド1
                         padding=1)     # パディングのサイズは1
    # vgg6の畳み込み層1
    conv6 = nn.Conv2d(512,  # 入力時のチャネル数
                      1024, # 出力時のチャネル数(フィルター数)
                      kernel_size=3,# フィルターサイズ3×3
                      padding=6,    # パディングのサイズは6
                      dilation=6)   # 畳み込みのポイント間の間隔を6にする
    # vgg6の畳み込み層2
    conv7 = nn.Conv2d(1024, # 入力時のチャネル数
                      1024, # 出力時のチャネル数(フィルター数)
                      kernel_size=1) # フィルターサイズ1×1
    # vgg5のプーリング層、vgg6の畳み込み層1と畳み込み層2をlayersに追加
    layers += [pool5,
               conv6, nn.ReLU(inplace=True), # 畳み込みの活性化はReLU
               conv7, nn.ReLU(inplace=True)] # 畳み込みの活性化はReLU
    
    # リストlayersをnn.ModuleListに格納してReturnする
    return nn.ModuleList(layers)

In [3]:
vgg = make_vgg()

In [4]:
sample = torch.randn((4, 3, 300, 300))
with torch.no_grad():
    for i, layer in enumerate(vgg):
        sample = layer(sample)
        if layer.__class__ != nn.modules.activation.ReLU:
            print(f"【{layer}】 ==> {sample.shape}")

【Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 64, 300, 300])
【Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 64, 300, 300])
【MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)】 ==> torch.Size([4, 64, 150, 150])
【Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 128, 150, 150])
【Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 128, 150, 150])
【MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)】 ==> torch.Size([4, 128, 75, 75])
【Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 256, 75, 75])
【Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 256, 75, 75])
【Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 256, 75, 75])
【MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1,

最終的に1024チャンネル、19x19の特徴量マップになる  
vgg4までの特徴量マップはL2正則化されて、利用される  
最後のvgg6まで通ったあとには、extrasモジュールによって再度畳み込みが実施される

# 追加のレイヤー(extras)

In [5]:
def make_extras():
    '''
    Returns：
      (nn.ModuleList): extrasのモジュール(部品)のリスト
    '''
    layers = []        # モジュールを格納するリスト
    in_channels = 1024 # vggから出力される画像データのチャネル数

    # vggに配置する畳み込み層のフィルター数(チャネル数に相当)
    cfg = [256, 512,  # extras1
           128, 256,  # extras2
           128, 256,  # extras3
           128, 256]  # extras4
    
    # extras1
    # 出力の形状:(バッチサイズ, 512, 10, 10)
    layers += [nn.Conv2d(in_channels,      # 入力時のチャネル数(1024)
                         cfg[0],           # 出力時のチャネル数(256) 
                         kernel_size=(1))] # フィルターサイズ1×1
    layers += [nn.Conv2d(cfg[0],           # 入力時のチャネル数(256)
                         cfg[1],           # 出力時のチャネル数(512) 
                         kernel_size=(3),  # フィルターサイズ3×3
                         stride=2,         # ストライドは2
                         padding=1)]       # パディングのサイズは1
    
    # extras2    
    # 出力の形状:(バッチサイズ, 256, 5, 5)
    layers += [nn.Conv2d(cfg[1],           # 入力時のチャネル数(512)
                         cfg[2],           # 出力時のチャネル数(128) 
                         kernel_size=(1))] # フィルターサイズ1×1
    layers += [nn.Conv2d(cfg[2],           # 入力時のチャネル数(128)
                         cfg[3],           # 出力時のチャネル数(256) 
                         kernel_size=(3),  # フィルターサイズ3×3
                         stride=2,         # ストライドは2
                         padding=1)]       # パディングのサイズは1
    
    # extras3
    # 出力の形状:(バッチサイズ, 256, 3, 3)
    layers += [nn.Conv2d(cfg[3],           # 入力時のチャネル数(256)
                         cfg[4],           # 出力時のチャネル数(128)
                         kernel_size=(1))]
    layers += [nn.Conv2d(cfg[4],           # 入力時のチャネル数(128)
                         cfg[5],           # 出力時のチャネル数(256)
                         kernel_size=(3))] # フィルターサイズ3×3
    
    # extras4
    # 出力の形状:(バッチサイズ, 256, 1, 1)
    layers += [nn.Conv2d(cfg[5],           # 入力時のチャネル数(256)
                         cfg[6],           # 出力時のチャネル数(128)
                         kernel_size=(1))]
    layers += [nn.Conv2d(cfg[6],           # 入力時のチャネル数(128)
                         cfg[7],           # 出力時のチャネル数(256)
                         kernel_size=(3))] # フィルターサイズ3×3

    # リストlayersをnn.ModuleListに格納してReturnする
    return nn.ModuleList(layers)

In [6]:
extras = make_extras()

In [7]:
sample = torch.randn((4, 1024, 19, 19))
with torch.no_grad():
    for i, layer in enumerate(extras):
        sample = layer(sample)
        if layer.__class__ != nn.modules.activation.ReLU:
            print(f"【{layer}】 ==> {sample.shape}")

【Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))】 ==> torch.Size([4, 256, 19, 19])
【Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))】 ==> torch.Size([4, 512, 10, 10])
【Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))】 ==> torch.Size([4, 128, 10, 10])
【Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))】 ==> torch.Size([4, 256, 5, 5])
【Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))】 ==> torch.Size([4, 128, 5, 5])
【Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))】 ==> torch.Size([4, 256, 3, 3])
【Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))】 ==> torch.Size([4, 128, 3, 3])
【Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))】 ==> torch.Size([4, 256, 1, 1])


extrasはvgg6の出力をinputとして、最終的に256チャンネル、1x1の特徴量マップとなる  
extrasは大きな物体を検出に利用される  
小さな物体は畳み込みの回数が少ないので検出性能が悪くなる傾向にある

<img src="https://cvml-expertguide.net/wp-content/uploads/2022/11/d345a0950fcf9161a425919d0b8366ef.png" width="500" />

一つの特徴量に一つのデフォルトボックスを用意すると合計が1940個になる

- vgg4(out1):38x38
- vgg6(out2):19x19
- extras1(out3):10x10
- extras2(out4):5x5
- extras3(out5):3x3
- extras4(out6):1x1

<img src="https://www.researchgate.net/publication/327077443/figure/fig5/AS:959259466539037@1605716687861/The-idea-of-default-boxes-applied-in-SSD-For-each-default-box-the-offsets-and.png" width=500>

実際はアスペクト比の異なるデフォルトボックスを複数用意するので合計は8732個になる

# デフォルトボックスの位置補正用のレイヤー

In [8]:
def make_loc(dbox_num=[4, 6, 6, 6, 4, 4]):
    ''' デフォルトボックスのオフセットを出力するlocネットワークを生成
    
    Parameters:
      dbox_num(intのリスト):
          out1～out6それぞれに用意されるデフォルトボックスの数
    Returns：
      (nn.ModuleList): extrasのモジュール(部品)のリスト
    '''
    # ネットワークのモジュールを格納するリスト
    loc_layers = []
    # vgg4の畳み込み層3からの出力にL2Normでの正規化の処理を適用した
    # out1に対する畳み込み層1
    loc_layers += [nn.Conv2d(512,           # 入力時のチャネル数
                             dbox_num[0]*4, # 出力時のチャネル数16
                             kernel_size=3, # フィルターサイズ3×3
                             padding=1)]    # パディングのサイズは1

    # vgg6からの最終出力out2に対する畳み込み層2
    loc_layers += [nn.Conv2d(1024,          # 入力時のチャネル数
                             dbox_num[1]*4, # 出力時のチャネル数24
                             kernel_size=3, # フィルターサイズ3×3
                             padding=1)]    # パディングのサイズは1

    # extrasのext1からの出力out3に対する畳み込み層3
    loc_layers += [nn.Conv2d(512,           # 入力時のチャネル数
                             dbox_num[2]*4, # 出力時のチャネル数24
                             kernel_size=3, # フィルターサイズ3×3
                             padding=1)]    # パディングのサイズは1

    # extrasのext2からの出力out4に対する畳み込み層4
    loc_layers += [nn.Conv2d(256,           # 入力時のチャネル数
                             dbox_num[3]*4, # 出力時のチャネル数24
                             kernel_size=3, # フィルターサイズ3×3
                             padding=1)]    # パディングのサイズは1

    # extrasのext3からの出力out5に対する畳み込み層5
    loc_layers += [nn.Conv2d(256,           # 入力時のチャネル数
                             dbox_num[4]*4, # 出力時のチャネル数16
                             kernel_size=3, # フィルターサイズ3×3
                             padding=1)]    # パディングのサイズは1

    # extrasのext4からの出力out6に対する畳み込み層6
    loc_layers += [nn.Conv2d(256,           # 入力時のチャネル数
                             dbox_num[5]*4, # 出力時のチャネル数16
                             kernel_size=3, # フィルターサイズ3×3
                             padding=1)]    # パディングのサイズは1

    # リストloc_layersをnn.ModuleListに格納してReturnする
    return nn.ModuleList(loc_layers)

In [9]:
loc = make_loc()

In [10]:
with torch.no_grad():
    for i, layer in enumerate(loc):
        if i == 0:
            sample = torch.randn((4, 512, 38, 38))
        elif i == 1:
            sample = torch.randn((4, 1024, 19, 19))
        elif i == 2:
            sample = torch.randn((4, 512, 10, 10))
        elif i == 3:
            sample = torch.randn((4, 256, 5, 5))
        elif i == 4:
            sample = torch.randn((4, 256, 3, 3))
        elif i == 5:
            sample = torch.randn((4, 256, 1, 1))
            
        sample = layer(sample)
        if layer.__class__ != nn.modules.activation.ReLU:
            print(f"【{layer}】 ==> {sample.shape}")

【Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 16, 38, 38])
【Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 24, 19, 19])
【Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 24, 10, 10])
【Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 24, 5, 5])
【Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 16, 3, 3])
【Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 16, 1, 1])


locはデフォルトボックスの位置を学習する  
out1、out3、out5には4つのデフォルトボックスを用意して、out2、out4、out6には6つのデフォルトボックスを用意する  
デフォルトボックス毎に4次元(中心位置x, 中心位置y, 幅, 高さ)を出力するので  
例えばout1は4x4=16のチャンネルになる  

# クラス確率出力用のレイヤー

In [11]:
def make_conf(classes_num=21, dbox_num=[4, 6, 6, 6, 4, 4]):
    ''' デフォルトボックスに対する各クラスの確率を出力するネットワークを生成
    
    Parameters:
      class_num(int): クラスの数
      dbox_num(intのリスト):
        out1～out6それぞれに用意されるデフォルトボックスの数
        
    Returns：
      (nn.ModuleList): extrasのモジュール(部品)のリスト
    '''
    # ネットワークのモジュールを格納するリスト
    conf_layers = []

    # vgg4の畳み込み層3からの出力にL2Normでの正規化の処理を適用した
    # out1に対する畳み込み層1
    conf_layers += [nn.Conv2d(512,          # 入力時のチャネル数
                              dbox_num[0]*classes_num, # 出力時は84
                              kernel_size=3,# フィルターサイズ3×3
                              padding=1)]   # パディングのサイズは1

    # vgg6からの最終出力out2に対する畳み込み層2
    conf_layers += [nn.Conv2d(1024,         # 入力時のチャネル数
                              dbox_num[1]*classes_num, # 出力時は126
                              kernel_size=3,# フィルターサイズ3×3
                              padding=1)]   # パディングのサイズは1

    # extrasのext1からの出力out3に対する畳み込み層3
    conf_layers += [nn.Conv2d(512,          # 入力時のチャネル数
                              dbox_num[2]*classes_num, # 出力時は126
                              kernel_size=3,# フィルターサイズ3×3
                              padding=1)]   # パディングのサイズは1

    # extrasのext2からの出力out4に対する畳み込み層4
    conf_layers += [nn.Conv2d(256,          # 入力時のチャネル数
                              dbox_num[3]*classes_num, # 出力時は126
                              kernel_size=3,# フィルターサイズ3×3
                              padding=1)]   # パディングのサイズは1

    # extrasのext3からの出力out5に対する畳み込み層5
    conf_layers += [nn.Conv2d(256,          # 入力時のチャネル数
                              dbox_num[4]*classes_num, # 出力時は84
                              kernel_size=3,# フィルターサイズ3×3
                              padding=1)]   # パディングのサイズは1

    # extrasのext4からの出力out6に対する畳み込み層6
    conf_layers += [nn.Conv2d(256,          # 入力時のチャネル数
                              dbox_num[5]*classes_num, # 出力時は84
                              kernel_size=3,# フィルターサイズ3×3
                              padding=1)]   # パディングのサイズは1

    # リストconf_layersをnn.ModuleListに格納してReturnする
    return nn.ModuleList(conf_layers)


In [12]:
conf = make_conf()

In [13]:
with torch.no_grad():
    for i, layer in enumerate(conf):
        if i == 0:
            sample = torch.randn((4, 512, 38, 38))
        elif i == 1:
            sample = torch.randn((4, 1024, 19, 19))
        elif i == 2:
            sample = torch.randn((4, 512, 10, 10))
        elif i == 3:
            sample = torch.randn((4, 256, 5, 5))
        elif i == 4:
            sample = torch.randn((4, 256, 3, 3))
        elif i == 5:
            sample = torch.randn((4, 256, 1, 1))
            
        sample = layer(sample)
        if layer.__class__ != nn.modules.activation.ReLU:
            print(f"【{layer}】 ==> {sample.shape}")

【Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 84, 38, 38])
【Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 126, 19, 19])
【Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 126, 10, 10])
【Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 126, 5, 5])
【Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 84, 3, 3])
【Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))】 ==> torch.Size([4, 84, 1, 1])


confはデフォルトボックス毎のクラス確率を出力する  
例えばout1はクラスの数(21)xデフォルトボックス(4)=84フィルタ数になる

# デフォルトのバウンディングボックス生成

In [14]:
from itertools import product as product
from math import sqrt as sqrt

class DBox(object):
    '''8732個のDBoxの(x座標, y座標, 幅, 高さ)を生成する
    
    Attributes:
      image_size(int): イメージのサイズ
      feature_maps(list): out1～out6における特徴量マップのサイズのリストを保持
      num_priors(int): feature_mapsの要素数、out1～out6の個数6を保持
      steps(list): DBoxのサイズのリストを保持
      min_sizes(list): 小さい正方形のDBoxのサイズを保持
      max_sizes(list): 大きい正方形のDBoxのサイズを保持
      aspect_ratios(list): 長方形のDBoxのアスペクト比を保持
    '''
    def __init__(self, cfg):
        '''インスタンス変数の初期化を行う
        '''
        super(DBox, self).__init__() # スーパークラスのコンストラクターを実行

        # 画像サイズ(300)を設定
        self.image_size = cfg['input_size']
        # out1～out6における特徴量マップのサイズ[38, 19, …]を設定
        self.feature_maps = cfg['feature_maps']
        # out1～out6の個数=6を設定
        self.num_priors = len(cfg["feature_maps"])
        # DBoxのサイズ[8, 16, 32, …]を設定 
        self.steps = cfg['steps']
        # 小さい正方形のDBoxのサイズ[30, 60, 111, …] を設定
        self.min_sizes = cfg['min_sizes']
        # 大きい正方形のDBoxのサイズ[60, 111, 162, …] を設定
        self.max_sizes = cfg['max_sizes']
        # 長方形のDBoxのアスペクト比[[2],[2,3],[2,3], ...]を設定
        self.aspect_ratios = cfg['aspect_ratios']

    def make_dbox_list(self):
        '''DBoxを作成する
        
        Returns:
          (Tensor)DBoxの[cx, cy, width, height]を格納した(8732, 4)の形状のテンソル
        '''
        mean = []
        # out1～out6における特徴量マップの数(6)だけ繰り返す
        # 特徴量マップのサイズのリストからインデックスをk、サイズをfに取り出す
        # 'feature_maps': [38, 19, 10, 5, 3, 1]
        # k: 0,1,2,3,4,5
        # f: 38, 19, 10, 5, 3, 1
        for k, f in enumerate(self.feature_maps):
            # fまでの数をrepeat=2を指定して2つのリストにして組み合わせ(直積)を作る
            # f=38の場合
            # i: 0,0,0,0,... の38個の0に対して
            # j: 0,1,2,3, ..., 37を組み合わせる
            # (i,j)は(0,0)(0,1)...(0,37)~(37,0)...(37,37)
            for i, j in product(range(f), repeat=2):  
                # 特徴量の画像サイズをDBoxのサイズsteps[k]で割る(kはインデックス)
                # 300 / 'steps': [8, 16, 32, 64, 100, 300]
                f_k = self.image_size / self.steps[k]

                # 特徴量ごとのDBoxの中心のx座標、y座標を求める
                # (0～1の範囲に規格化)
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # 小さい正方形のDBoxのサイズmin_sizes[k](kはインデックス)を
                # 画像のサイズで割る
                # 'min_sizes': [30, 60, 111, 162, 213, 264] / 300
                s_k = self.min_sizes[k]/self.image_size
                # 小さい正方形のDBoxの[cx,cy, width, height]をリストに追加
                mean += [cx, cy, s_k, s_k]

                # 大きい正方形のDBoxのサイズmin_sizes[k](kはインデックス)を
                # 画像のサイズで割る
                # max_sizes': [45, 99, 153, 207, 261, 315] / 300
                # さらにs_kを掛けて平方根を求める
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                # 大きい正方形のDBoxの[cx,cy, width, height]をリストに追加
                mean += [cx, cy, s_k_prime, s_k_prime]

                # 長方形のDBoxの[cx,cy, width, height]をリストに追加
                for ar in self.aspect_ratios[k]:
                    # widthはs_kにアスペクト比の平方根を掛けたもの
                    # heightはs_kをアスペクト比と平方根で割ったもの
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    # widthはs_kをアスペクト比と平方根で割ったもの
                    # heightはs_kにアスペクト比の平方根を掛けたもの
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]

        # DBoxの[cx,cy, width, height]のリストを(8732, 4)の2階テンソルに変換
        output = torch.Tensor(mean).view(-1, 4)
        # DBoxの大きさが1を超えている場合は1にする
        output.clamp_(max=1, min=0)
        
        # DBoxの[cx,cy, width, height]を格納した2階テンソルを返す
        return output

# 全体のアーキテクチャ

In [23]:
import torch.nn.functional as F

In [21]:
from ssd import make_vgg, make_extras, L2Norm, DBox, Detect

In [22]:
class SSD(nn.Module):
    '''SSDモデルを生成するクラス
    
    Attributes:
      phase(str): 'train'または'test'
      classes_num(int): クラスの数
      vgg(object): vggネットワーク
      extras(object): extrasネットワーク
      L2Norm(object): L2norm層
      loc(object): locネットワーク
      conf(object): confネットワーク
      dbox_list(Tensor):
        DBoxの[cx, cy, width, height]を格納した(8732, 4)の形状のテンソル
      detect(object):
        Detectクラスのforward()を実行する関数オブジェクト
    '''
    def __init__(self, phase, cfg):
        super().__init__()
        
        # 動作モード(train or test)
        self.phase = phase
        # クラス数
        self.classes_num = cfg["classes_num"]
        
        self.vgg = make_vgg()
        self.extra = make_extras()
        self.l2norm = L2Norm()
        
        # dbox_numはout1~out6までに用意するデフォルトボックスの数
        self.loc = make_loc(cfg["dbox_num"])
        self.conf = make_conf(cfg["classes_num"], cfg["dbox_num"])
        
        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()
        
        # 推論で利用する場合
        if phase == "test":
            self.detect = Detect.appy
    
    def forward(self, x):
        """
        x: (バッチサイズ, 3, 300, 300)
        """
        
        # 出力を保存するリスト
        out_list, loc, conf = [], [], []
        
        # vggネットワークinputを通す(index23は4層まで)
        for k in range(23):
            x = self.vgg[k](x)
            
        # vgg4の出力をL2Normに通して保存
        out1 = self.l2norm(x)
        out_list.append(out1)
        
        # vgg残りの層を通す
        for k in range(23, len(self.vgg)):
            x = self.vgg[k](x)
        
        # vgg6の出力をappend
        out_list.append(x)
        
        for k, v in enumerate(self.extras):
            x = F.relu(v(x), inpalce=True)
            if k % 2 == 1:
                # out3-out6を追加
                out_list.append(x)
        
        # out1-out6を位置情報レイヤーとクラス分類レイヤーに渡す
        for (x, l, c) in zip(out_list, self.loc, self.conf):
            # (バッチサイズ, オフセット値4*DBoxの種類, 特徴量(h),特徴量(w))から
            # (バッチサイズ, 特徴量(h), 特徴量(w), オフセット値4*DBoxの種類)の変換
            loc.append(l(x).permute(0, 2, 3, 1).contiguous())
            conf.append(c(x).permute(0, 2, 3, 1).contiguous())
            
        # flat化
        # loc:(バッチサイズ, 34928)
        # conf:(バッチサイズ, 183372)
        loc = torch.cat([o.view(o.size(0), -1) for o in loc], 1)
        conf = torch.cat([o.view(o.size(0), -1) for o in conf], 1)
        
        # デフォルトボックスx4つのオフセット値のshapeにする
        # (バッチサイズ, 8732, 4)
        loc = loc.view(loc.size(0), -1, 4)
        # デフォルトボックスxクラス数のshapeにする
        # (バッチサイズ, 8732, 21)
        conf = conf.view(conf.size(0), -1, self.classes_num)
        
        # outputとしてまとめる
        output = (loc, conf, self.dbox_list)
        
        if self.phase == "test":
            # 推論時は確信度と位置情報を後処理して
            # (バッチサイズ, 21(クラス), 200(Top200のBBox), 5) としてreturnされる
            # 最後の次元の5は[BBoxの確信度, xmin, ymin, width, height]
            self.detect(*output)
        else:
            return output